<a href="https://colab.research.google.com/github/seschm/Internship-Gaertner/blob/main/MLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp
import random as rng
from tqdm.auto import tqdm
from numpy import sqrt, cos, sin, exp, pi, log2
from scipy.stats import unitary_group
from scipy.stats import norm

In [ ]:
def MLE(nQubits,calibration_states,noisy_probabilities,POVM_ideal,p,samplesize):
    '''
    calibration_states: axis 0 = state, axis 1 = row, axis 2 = column
    noisy_probabilities: axis 0 = state, axis 1 = probabilities
    POVM_ideal / _reconstructed: axis 0 = element, axis 1 = row, axis 2 = column
    numb_of_detections: axis 0 = POVM_element, axis 1 = states, values = counts
    p_ideal: axis 0 = POVM_element, axis 1 = states, values = current ideal probability
    fracs: axis 0 = POVM_element, axis 1 = states, values = fractions

    '''

    numb_of_detections = noisy_probabilities.T * samplesize

    loop_num = 0
    loop_max = 2*10**3
    dist = 1
    dist_min = 10**-9

    POVM_reconstructed = np.array([p/2**nQubits*np.eye(2**nQubits) + (1-p)*POVM_elem for POVM_elem in POVM_ideal])

    while loop_num < loop_max and dist > dist_min:

        p_ideal = np.array([perform_multi_qubit_measurement(state,POVM_reconstructed) for state in calibration_states]).T
        fracs = numb_of_detections/p_ideal

        G = np.einsum('lm,ln,nij,ljk,mko->io',fracs,fracs,calibration_states,POVM_reconstructed,calibration_states)

        eig_vals,U = sp.linalg.eig(G)
        l_sqrt_inv = np.diag(1/np.sqrt(eig_vals))
        l_inv = U@l_sqrt_inv@U.conj().T # -> why not as in paper: U_dagger L_sqrt U ???

        R = np.einsum('ij,lm,mjk->lik',l_inv,fracs,calibration_states)

        POVM_reconstruction_old = POVM_reconstructed
        #POVM_reconstructed = np.einsum('lij,ljk,lkm->lim',R,POVM_reconstructed,R.conj().T)
        POVM_reconstructed = np.einsum('lij,ljk,lmk->lim',R,POVM_reconstructed,R.conj())

        dist = np.sum([np.linalg.norm(POVM_reconstructed-POVM_reconstruction_old)])

        if loop_num % 50 == 0:
            print(f'loop number: {loop_num}, distance: {dist}')

        loop_num += 1

    print(f'final loop number: {loop_num}, final distance: {dist}')

    return POVM_reconstructed, dist

In [ ]:
def sampling(probabilities,samplesize):
    """
    Takes in a probability distribution and samples from it.
    """
    sampling = rng.choices(np.arange(0,len(probabilities)), weights=probabilities, k=samplesize)
    sampled_probabilities = []
    for element in np.arange(0,len(probabilities)):
        sampled_probabilities.append(sampling.count(element)/samplesize)
    return sampled_probabilities

In [ ]:
def perform_noisy_multi_qubit_measurement(state,POVM,p):
    """
    Generates probabilities of the noisy state.
    First applies the noise, then generates the probabilities.
    """
    noisy_state = depolarizing_channel(state,p)
    return perform_multi_qubit_measurement(noisy_state,POVM)

In [ ]:
def perform_multi_qubit_measurement(state,POVM):
    """
    Performs a multi qubit measurement with the given POVM.
    """
    probabilities = []
    for element in POVM:
        probabilities.append(np.trace(element@state))
    return probabilities

In [ ]:
def generate_POVM(theta,phi,nQubit):
    """
    Generates a POVM consisting of multi qubit projectors on to the axis defined by the two angles theta and phi.
    First generates all possible states (combinations of spin up and down) as strings containing 1s and 0s.
    The order is equal to binary counting.
    Then generates the single qubit projector and its orthogonal projector.
    In the next step all single qubit projectors corresponding to the same state are gathered in the correct order and then tensored together to get an element of the POVM.
    Returns the POVM with the elements beeing ordered equal to binary counting (-> up ... up, up ... up down, up ... up down up, ...).
    """
    up_and_downs = []

    for i in range(2**nQubit):
        binary = bin(i)[2:]
        zeros = np.zeros(nQubit-len(binary), dtype=int)
        for k in range(nQubit-len(binary)):
            binary = '0' + binary
        up_and_downs.append(binary)

    projector=generate_single_qubit(theta,phi)
    orthogonal_projector = np.eye(2)-projector
    POVM = []

    for i in range(2**nQubit):
        tensored_projector = []
        single_projectors = []
        for j in range(nQubit):
            if up_and_downs[i][j] == '0':
                single_projectors.append(projector)
            if up_and_downs[i][j] == '1':
                single_projectors.append(orthogonal_projector)
        tensored_projector.append(single_projectors[0])
        for k in range(nQubit-1):
            tensored_projector.append(np.kron(tensored_projector[-1],single_projectors[k+1]))
        POVM.append(tensored_projector[-1])

    return POVM

In [ ]:
def depolarizing_channel(state,p):
    """
    Applies a depolarizing channel to the given state with p the probability of the completely mixed state.
    """
    d=len(state[0])
    return p*np.eye(d)/d+(1-p)*state

In [ ]:
def generate_random_seperable_pure_state(nQubit):
    """
    Generates random seperable pure state.
    First generate the desired number of random pure states.
    Then tensor them together.
    """
    single_qubits = []
    for i in range(0,nQubit):
        single_qubits.append(generate_random_pure_state(1))
    tensored_qubits = [single_qubits[0]]
    for i in range(1,nQubit):
        tensored_qubits.append(np.kron(tensored_qubits[-1],single_qubits[i]))
    return tensored_qubits[-1]

In [ ]:
def generate_single_qubit(theta,phi):
    """
    Generates single qubit out of the given angles theta and phi.
    First construct the single qubit state as an array of shape (2, 1).
    Then compute the matrixproduct with its adjoint state.
    """
    state = np.array([[cos(theta/2)],[sin(theta/2)*exp(phi*1.j)]])
    return state@state.conj().T

In [ ]:
def generate_random_pure_state(nQubit):
    """
    Generates Haar random pure state.
    To generate a random pure state, take any basis state, e.g. |00...00>
    and apply a random unitary matrix. For consistency each basis state should be the same.
    """
    baseRho=np.zeros((2**nQubit,2**nQubit),dtype=complex)
    baseRho[0,0]=1
    U=unitary_group.rvs(2**nQubit)
    return U@baseRho@U.conj().T

In [ ]:
def accuracy_MSE(acc_input, acc_target_output,nQubit):
  acc = np.mean(np.sum(np.abs(acc_input-acc_target_output)**2,axis=2)/(2**nQubit))
  return acc

In [ ]:
def accuracy_KLD(acc_input, acc_target_output):
  acc = np.mean(np.sum(acc_target_output*np.log(acc_target_output/acc_input),axis=2))
  return acc

In [ ]:
def accuracy_IF(acc_input, acc_target_output):
  acc = np.mean(1-(np.sum(np.sqrt(acc_target_output*acc_input),axis=2))**2)
  return acc

In [ ]:
nQubit = 2
training_states = 100
theta = 0
phi = 0
p = 0.15
samplesize = 1000

POVM = generate_POVM(theta,phi,nQubit)
calib = []
pro = []

for i in range(training_states):
    state = generate_random_seperable_pure_state(nQubit)
    calib.append(state)
    noisy_probs = perform_noisy_multi_qubit_measurement(state,POVM,p)
    noisy_probs_sampled = sampling(noisy_probs,samplesize)
    pro.append(noisy_probs_sampled)


calibration_states = np.asarray(calib)
noisy_probabilities = np.asarray(pro)

P, dist = MLE(nQubit,calibration_states,noisy_probabilities,POVM,p,samplesize)

/usr/lib/python3.10/random.py:536: ComplexWarning: Casting complex values to real discards the imaginary part
  if not _isfinite(total):


loop number: 0, distance: 0.020856877517224516
loop number: 50, distance: 9.158349319907202e-05
loop number: 100, distance: 1.030466355050956e-05
loop number: 150, distance: 1.5251826406592527e-06
loop number: 200, distance: 2.547408036114234e-07
loop number: 250, distance: 4.533249224739167e-08
loop number: 300, distance: 8.358183990540306e-09
loop number: 350, distance: 1.5769797525740386e-09
final loop number: 365, final distance: 9.91901444674732e-10


In [ ]:
def linear_inversion(nQubit,calibration_states,noisy_probabilities,POVM,p,samplesize):
    '''
    #,probabilities,test_input,test_target_output
    '''

    #POVMs = []

    #for i in range(len(states)):
    #    P, dist = MLE(nQubit,calibration_states[i],noisy_probabilities[i],POVM,p,samplesize)
    #    POVMs.append(P)

    #POVM_recon = []
    POVM_recon, dist = MLE(nQubit,calibration_states,noisy_probabilities,POVM,p,samplesize)

    L = np.zeros((2**nQubit,2**nQubit))

    #print(np.shape(L))
    #print(np.shape(POVM_recon))
    #print(POVM_recon)
    for i in range(2**nQubit):
        for j in range(2**nQubit):
            L[j][i] = POVM_recon[i][j][j]

    L_inv = np.linalg.inv(L)
    print(np.shape(L_inv))
    print(np.shape(noisy_probabilities))

    #probs_cor = np.einsum('ij,klj->kli',L_inv,test_input)
    probs_cor_tr = np.einsum('ij,lj->li',L_inv,noisy_probabilities)

    #acc_MSE = accuracy_MSE(test_input, test_target_output,nQubit)
    #acc_MSE_tr = accuracy_MSE(probs_cor, probabilities,nQubit)

    #acc_KLD = accuracy_KLD(test_input, test_target_output,nQubit)
    #acc_KLD_tr = accuracy_KLD(probs_cor, probabilities,nQubit)

    #acc_IF = accuracy_IF(test_input, test_target_output,nQubit)
    #acc_IF_tr = accuracy_IF(probs_cor, probabilities,nQubit)

    return probs_cor_tr#probs_cor, [acc_MSE,acc_MSE_tr], [acc_KLD,acc_KLD_tr], [acc_IF,acc_IF_tr]